In [1]:
import json 
import os 
import cv2
from pprint import pprint
import requests
import numpy as np
import random
import math
import albumentations as A
from tqdm import tqdm
import uuid
import requests
from matplotlib import pyplot as plt
%matplotlib inline

/home/hbdesk/labelstudio_convert/venv/lib/python3.12/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
LABEL_STUDIO_KEY="eaffb8f4719efb17f9b227ed56fee991c4d06a0a"
LABEL_STUDIO_URL="http://192.168.231.52:8080/"
OCR_SERVER="http://localhost:3000/en"


def string_from_img(image_path: str):
    with open(image_path, "rb") as image_file:
        files = { "file": (image_path, image_file, "image/jpeg") }
        response = requests.post(OCR_SERVER, files=files)
        if response.status_code==200:
            res_string = response.json()["string"]
            return res_string
        else:
            return None

def xywh2xyxy(xywh, original_width, original_height):
    """
    Convert [x, y, w, h] in percentage format to xyxy absolute coordinates.

    Parameters:
    xywh (list): List of [x, y, w, h] as percentages, where x and y represent the top-left corner.
    original_width (int): Original width of the image.
    original_height (int): Original height of the image.

    Returns:
    tuple: (x_min, y_min, x_max, y_max) in absolute coordinates.
    """
    x_percent, y_percent, width_percent, height_percent = xywh

    x_min = (x_percent / 100) * original_width
    y_min = (y_percent / 100) * original_height
    x_max = x_min + (width_percent / 100) * original_width
    y_max = y_min + (height_percent / 100) * original_height

    return x_min, y_min, x_max, y_max

def get_img_from_studio(image_path, save_path="./weapon_img"):
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    download_url = f"{LABEL_STUDIO_URL}{image_path}"
    headers = {
        'Authorization': f'Token {LABEL_STUDIO_KEY}'
    }
    response = requests.get(download_url, headers=headers)
    if response.status_code == 200:
        filename = os.path.basename(image_path)
        save_path = os.path.join(save_path, filename)
        with open(save_path, 'wb') as f:
            f.write(response.content)
            return save_path
        print(f"Image successfully downloaded as {filename}")
    else:
        print(f"Failed to download image. Status code: {response.status_code}")
        return None

In [2]:
with open("./char_dset.json", "r") as loadjson:
    data = json.loads(loadjson.read())
pprint(data[0])

NameError: name 'json' is not defined

In [ ]:
final_bboxes = []
for annotation in tqdm(data):
    # print(len(annotation["annotations"][0]["result"]))
    char_bboxes = []
    if len(annotation["annotations"][0]["result"]) > 0:
        db_fname = annotation["data"]["image"]
        width = annotation["annotations"][0]["result"][0]["original_width"]
        height = annotation["annotations"][0]["result"][0]["original_height"]
        image_name = get_img_from_studio(db_fname, "/home/hbdesk/labelstudio_convert/char_download/")
        for _bbox in annotation["annotations"][0]["result"]:
            xywh = [_bbox["value"]["x"], _bbox["value"]["y"], _bbox["value"]["width"], _bbox["value"]["height"]]
            x1 , y1 , x2, y2  = xywh2xyxy(xywh, width, height)
            char_bboxes.append([x1,y1,x2,y2])
    bbox_image_pair = {image_name: char_bboxes}
    final_bboxes.append(bbox_image_pair)
print(final_bboxes[0])

In [ ]:
stacked = []
strings = []
for bboxes in final_bboxes:
    string_img = string_from_img(list(bboxes.keys())[0])
    string_img = string_img.replace(" ", "")
    strings.append(string_img)
    img = cv2.imread(list(bboxes.keys())[0])
    stack1 = []
    for bbox in bboxes:
        boxes = bboxes[bbox]
        for box in boxes:
            x1,y1,x2,y2 = [int(x) for x in box]
            crop = img[y1:y2, x1:x2]
            h, w ,c = crop.shape
            pad_width = max(0, 24 - w)
            pad_height = max(0, 24 - h)
            top = pad_height // 2
            bottom = pad_height - top
            left = pad_width // 2
            right = pad_width - left
            _pad = np.pad(crop, ((top, bottom), (left, right), (0, 0)), mode='constant', constant_values=0)
            # _resize = cv2.cvtColor(cv2.resize(crop, (112, 112)), cv2.COLOR_BGR2GRAY)
            _pad = cv2.cvtColor(_pad, cv2.COLOR_BGR2GRAY)
            stack1.append(_pad)
            cv2.imwrite(f"chars/{uuid.uuid4().hex}.png", _pad)
    stacked.append(stack1)

In [ ]:
_set_string = set(strings)
chars = sorted(set("".join(_set_string)))
dest = "/home/hbdesk/labelstudio_convert/chars"
for char in chars:
    _dst = os.path.join(dest, char)
    if not os.path.exists(_dst):
        os.makedirs(_dst)

In [ ]:
INDEX = 200
print(" ".join(strings[INDEX]))
pprint(data[200])
plt.imshow(np.hstack(stacked[INDEX]))
plt.show()

In [ ]:
for _stridx, _string in enumerate(strings):
    img_list = stacked[_stridx]
    for _charidx, _char in enumerate(_string):
        dest_save_path = os.path.join(dest, _char)
        save_fname = f"{dest_save_path}/{uuid.uuid4().hex}.png"
        try:
            cv2.imwrite(save_fname, img_list[_charidx])
        except:
            pass